# Treetime Analysis

## Module Imports

In [17]:
from Bio import Phylo
from treetime.utils import parse_dates
import pandas

## Input File Paths

In [15]:
'''
tree_path = str(snakemake.input.tree)
aln_path = str(snakemake.input.snp_aln)
metadata_path = str(snakemake.input.metadata)
treetime_path = str(snakemake.output.treetime)
'''

tree_path = "../../results/iqtree/assembly/iqtree.core-chromosome.filter0.treefile"
metadata_path = "../../results/treetime/assembly/metadata.tsv"
print(tree_path)
print(metadata_path)

../../results/iqtree/assembly/iqtree.core-chromosome.filter0.treefile
../../results/treetime/assembly/metadata.tsv


## Constants and Variables

In [ ]:
name_column = "Accession"
dates_column = "BioSampleCollectionDate"
attribute = "BioSampleBiovar"
missing_data = "?"

### Parse Dates

In [18]:
states = pandas.read_csv(metadata_path, sep='\t')
if name_column in states.columns:
    taxon_name = name_column
if attribute in states.columns:
    attr = attribute

# Get tips names
tree = Phylo.read(tree_path, "nexus")
tree_tip_names = [t.name for t in tree.get_terminals()]

                             Accession BioSampleGeographicLocation  \
0  GCA_009669545.1_ASM966954v1_genomic                         USA   
1  GCA_009669555.1_ASM966955v1_genomic                         USA   
2  GCA_009669565.1_ASM966956v1_genomic                         USA   
3                            Reference                         USA   

   BioSampleCollectionDate  
0                     2000  
1                     2000  
2                     2000  
3                     2000  


In [12]:
with open(treetime_path, "w") as file:
    ...

NameError: name 'treetime_path' is not defined